In [ ]:
# QCB455/COS551 Final Project Kmer Rankings
# Author: Supraj Gunda
# Does not produce the final k-mer plot shown in final paper, but this attempt at graphing k-mers was made by Supraj.

# imports
from Bio import SeqIO
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def countKmers(seq, k):
    kmersCount = Counter()

    # so you don't start reading after sequence ends
    for i in range((len(seq) - k) + 1):
        # go through k-length windows and see if the sequence in the window matches possible kmers
        kmer = seq[i:(i + k)]
        kmersCount[kmer] += 1

    return kmersCount

In [ ]:
# rank the kmers in order of frequency
def rankKmers(fasta_files, k, topN=15):
    rankedData = []
    
    for fasta_file in fasta_files:
        genome_name = fasta_file.split('/')[-1]
        kmerCounter = Counter()
        
        # Count kmers in each genome from the fasta files
        for genome in SeqIO.parse(fasta_file, "fasta"):
            readableGenome = str(genome.seq)
            kmerCounter.update(countKmers(readableGenome, k))
        
        # extract the top 15 kmers, for example
        sorted = kmerCounter.most_common(topN)

        # must format like this so can transform into df easily
        for rank, (kmerIdentity, freq) in enumerate(sorted, start=1):
            rankedData.append({"Dataset": genome_name, "k-mer": kmerIdentity, "Rank": rank, "Count": freq})
    
    return pd.DataFrame(rankedData)

In [ ]:
def plotKmers(rankDF, topN=4):
    # create table
    rankPivot = rankDF.pivot(index="k-mer", columns="Dataset", values="Rank")
    rankPivot = rankPivot.index[:topN]
    
    # begin plot
    plt.figure(figsize=(12, 6))
    for kmer in rankPivot: 
        plt.plot(rankPivot.columns, rankPivot.loc[kmer], marker='o', label=kmer)
    
    plt.gca().invert_yaxis() 
    plt.xlabel("Dataset")
    plt.ylabel("Rank")
    plt.title(f"Top {topN} k-mer Rankings")
    plt.xticks(rotation=45, ha='right')
    plt.legend(title="k-mer", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    plt.show()

In [ ]:
# so you can add more genomes in the future if necessary
fastaFiles = [
    '/Users/Supraj1/qcb455/fastaFiles/GCA_Ash1.fna',
    '/Users/Supraj1/qcb455/fastaFiles/GCA_ASM.fna',
    '/Users/Supraj1/qcb455/fastaFiles/GCA_hg.fna',
    '/Users/Supraj1/qcb455/fastaFiles/GCF_GRCh38.fna',
    '/Users/Supraj1/qcb455/fastaFiles/GCF_T2T.fna']
k = 4
topN = 15

# can't combine all methods into one function as there will be not enough memory otherwise
rankDF = rankKmers(fastaFiles, k, topN)

# save ranks so that you can remove them from RAM - otherwise there will not be enough RAM to make the entire graph
rankDF.to_csv("kmerRanks.csv", index=False)

# plot
plotKmers(rankDF, topN)
